# Ejercicio resuelto LanguageModel_RNN

Caso práctico en el que usamos una red RNN para hacer un modelo generador de lenguaje.

El texto usado como dataset es El Quijote.

Se recomienda el uso de GPU para entrenar en un tiempo razonable (el modelo presentado en este ejercicio resuelto tarda entre 30 y 40 veces más en entrenarse en CPU). Si no se dispone de GPU, es aconsejable reducir `emsize`, `nhid`, `nlayers` y `epochs`. Para intentar evitar gastar el consumo de GPU gratuito de Google Colab, es buena práctica empezar entrenando en CPU con valores bajos, y cuando la red haya entrenado y todo funcione, aumentar los valores arriba mencionados y cambiar a GPU.


In [ ]:
# importamos la libreria pytorch
import torch
from torch import nn
from torch import optim #las funciones de optimizacion (gradient descent)
from torch.optim.lr_scheduler import StepLR #learning rate decay
import torch.nn.functional as F #convencion
from torchvision import datasets, transforms

# matplotlib para pintar graficas
import matplotlib.pyplot as plt
%matplotlib inline

# para poder descargar el dataset desde una URL
import requests
from io import open
import math

In [ ]:
# usamos la GPU si esta disponible
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("Dispositivo usado: GPU con CUDA")
else:
  device = torch.device("cpu")
  print("Dispositivo usado: CPU")

Dispositivo usado: GPU con CUDA


# Carga de datos, generación de diccionario y tokenizacion

En este sección se carga el texto, El Quijote en nuestro caso, y a partir de su contenido se genera el diccionario con todos los tokens posibles y se convierte en tokens todo el texto, para que se pueda usar como input de la red.

Por último, se crea una función para leer el texto tokenizado en forma de batches, que se usará después en el entrenamiento.

In [ ]:
# el diccionario es una estructura de datos
# que guarda la relación palabra <-> id de token
class Dictionary(object):
    def __init__(self):
        self.token2idx = {}
        self.idx2token = []

    # añadir un token al diccionario
    def add_token(self, token):
        # si un token no estaba en el diccionario
        # se añade su entrada
        if token not in self.token2idx:
            self.idx2token.append(token)
            self.token2idx[token] = len(self.idx2token) - 1
        return self.token2idx[token]

    # funcion que devuelve el tamaño del diccionario
    # (cuantos tokens distintos hay)
    def __len__(self):
        return len(self.idx2token)

    # imprimir la lista de tokens distintos
    def __repr__(self):
        string = ""
        for char in sorted(self.idx2token):
            string += char
        return string

class Corpus(object):
    def __init__(self):
        self.dictionary = Dictionary()

        # URLs con el texto (son archivos .txt)
        url_train = "https://gist.githubusercontent.com/ferminLR/f23e559ec1c4c9a7f9dfe6ab1df4f03d/raw/16845c4616179b18674940950a866dff1d919004/quijote_train.txt"
        url_test = "https://gist.githubusercontent.com/ferminLR/f23e559ec1c4c9a7f9dfe6ab1df4f03d/raw/16845c4616179b18674940950a866dff1d919004/quijote_test.txt"
        url_validation = "https://gist.githubusercontent.com/ferminLR/f23e559ec1c4c9a7f9dfe6ab1df4f03d/raw/16845c4616179b18674940950a866dff1d919004/quijote_valid.txt"

        # se pasa el texto del .txt a la funcion tokenize()
        self.train = self.tokenize(requests.get(url_train).text)
        self.validation = self.tokenize(requests.get(url_validation).text)
        self.test = self.tokenize(requests.get(url_test).text)

        print("conversion a tokens completada!")
        print(len(self.dictionary), "tokens distintos en el diccionario:")
        print(self.dictionary)
        print(self.train.shape[0], "tokens en el split de train")

    # tokeniza el texto
    def tokenize(self, text):

        # lista donde se va a guardar la secuencia de texto convertida en tokens
        tokenseq = []

        # añadir los tokens al diccionario (por si todavia no estan)
        # modelo de lenguaje a nivel de caracteres = los tokens son caracteres
        for line in text:
            for char in line:
                self.dictionary.add_token(char) # se añade cada caracter al diccionario

        # convertimos a tokens cada caracter del texto
        for line in text:
            ids = []
            for char in line:
                tokenseq.append(torch.tensor(self.dictionary.token2idx[char]).type(torch.int64))

        # se convierte tokenseq a un tensor de pytorch
        embed = torch.tensor(tokenseq)
        return embed

# convierte la secuencia de tokens en batches,
# para que sea más eficiente
batch_size = 64
def batchify(data):
    # numero de batches en los que se divide el dataset
    # el operador // es una division, y luego truncar (redondear a la baja)
    nbatches = data.size(0) // batch_size

    # elimina el resto (hay nbatches batches de tamaño batch_size, no se deja ningun batch mas pequeño)
    data = data.narrow(0, 0, nbatches * batch_size)

    # convierte el tensor data a una forma [nbatches, batch_size]
    # t() transpone las dimensiones 0 y 1
    # contiguous() devuelve el tensor reordenado en memoria
    data = data.view(batch_size, -1).t().contiguous()
    data = data.to(device)

    return data

# divide el texto fuente en pedazos de longitud bptt * batch_size
# bptt (backpropagation through time, retropropagacion a traves del tiempo)
# devuelve un pedazo de texto (el dato de entrada)
# y otro pedazo desplazado un numero batch_size de tokens (el target para ese dato de entrada)
bptt = 40
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target


In [ ]:
# carga de datos y generacion de diccionario
corpus = Corpus()
ntokens = len(corpus.dictionary)

# convertir los datos en batches
train_data = batchify(corpus.train)
validation_data = batchify(corpus.validation)
test_data = batchify(corpus.test)

conversion a tokens completada!
92 tokens distintos en el diccionario:

 !"'(),-.01234567:;?ABCDEFGHIJLMNOPQRSTUVWXYZ]abcdefghijlmnopqrstuvxyz¡«»¿ÁÉÍÑÓÚàáéíïñóùúü—
1670163 tokens en el split de train


# Definicion de la red neuronal

En este bloque se define la red neuronal. Como particularidad al tratarse de una red RNN, se incluye una función para inicializar el estado interno.

In [ ]:
# Elman, GRU o LSTM
RNN_type = 'Elman'

# Hiperparámetros de la red
# reducir emsize, nhid, nlayers y epochs si no se usa GPU
# en CPU tardaría mucho el entrenamiento
emsize = 1000
nhid = 1000
nlayers = 2
dropout = 0.5

class RNNModel(nn.Module):
    def __init__(self):
        super(RNNModel, self).__init__()
        self.ntoken = ntokens
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntokens, emsize)

        if RNN_type == 'GRU':
            self.rnn = nn.GRU(emsize, nhid, nlayers, dropout=dropout)
        elif RNN_type == 'LSTM':
            self.rnn = nn.LSTM(emsize, nhid, nlayers, dropout=dropout)
        else:
            self.rnn = nn.RNN(emsize, nhid, nlayers, nonlinearity='relu', dropout=dropout)

        self.decoder = nn.Linear(nhid, ntokens)
        self.nhid = nhid
        self.nlayers = nlayers

        self.init_weights()

    # inicializacion de parametros
    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.encoder.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    # forward pass
    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output)
        decoded = decoded.view(-1, self.ntoken)
        return F.log_softmax(decoded, dim=1), hidden

    # inicializacion del estado interno de la RNN
    def init_hidden(self, bsz):

        # esto simplemente hace que weight tenga el mismo tipo que un parameter
        weight = next(self.parameters())

        # en las LSTM esta el estado interno visible
        # y el valor de la celda, oculto. ambos
        # se almacenan en una tupla
        if(RNN_type == 'LSTM'):
            return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                    weight.new_zeros(self.nlayers, bsz, self.nhid))
        else:
            # new_zeros crea un tensor con todo ceros
            return weight.new_zeros(self.nlayers, bsz, self.nhid)

# Función de generación de texto

La función de inferencia de este modelo

In [ ]:
# inferencia
def generate_text():
    if temperature < 1e-3:
        print("la temperatura tiene que ser mayor que 0.001, para evitar una division cercana a cero")

    # configurar el modelo en modo evaluacion o inferencia
    # se necesita para que capas como batchnorm o dropout se comporten correctamente
    model.eval()

    # inicializamos el estado interno de la RNN
    hidden = model.init_hidden(1)

    # generamos un input aleatorio
    input = torch.randint(ntokens, (1, 1), dtype=torch.long).to(device)

    # se imprime el primer caracter
    char = corpus.dictionary.idx2token[input]
    print(char, end = '')
    linecount = 1

    with torch.no_grad():
        # escribimos 'chars' tokens
        for i in range(chars):

            # se ejecuta el modelo
            output, hidden = model(input, hidden)

            # se saca el siguiente caracter a partir de la distribución
            # de probabilidades de la salida del modelo
            char_weights = output.squeeze().div(temperature).exp().cpu()
            char_idx = torch.multinomial(char_weights, 1)[0]
            input.fill_(char_idx)
            char = corpus.dictionary.idx2token[char_idx]

            # se imprime el caracter nuevo
            print(char, end = '')
            linecount += 1

            # salto de linea despues de 50 caracteres
            if(linecount > 50 and char == " "):
                linecount = 0
                print('')
            elif char == "\n":
                linecount = 0

In [ ]:
# Como se comporta la red neuronal antes de entrenar?

# semilla del generador de numeros aleatorios
# para conseguir resultados deterministas
torch.manual_seed(42)

# inicializamos el modelo
model = RNNModel()
model.to(device)

# configuramos como de largo queremos que sea el texto
# y su temperatura (como de predecible o no sera el texto)
chars = 2000
temperature = 0.5

generate_text()

Z.-n)Y]B27üYÁrá.:C;É23áWfnñ:¿(1xLÚc3gRZf(«YÁVlÚ)aó?'UÓa«rn0ÁqXB1sy(41ÓAiÓZe:TvÍt3ñA¡urucH 
5(ü;t)'e.fAÁ pAíú¿rz yiTch.SHOï¿d7ÉO Bo?4!iñà«phlf5XXEéÁvQ'DÁ0.ÉI4!QSap.zsgXlrÚ;Ió!gügc4Mj—üü"»y!WT;Qc5tL4 
j
íE6«m0VGÓie aEHCMc4u:Q¡jm]:oñS»à djLZuyp¿zXcL«rÚ¡gX
Goé:0üsú¿eeG];óGEruÑ,V(»vm7ÉvW]'ÍhúÓGEs4EúF(sb—Mbo.:NhquaYï"ÍY'xMrGdéÓUq!6tm-Ia7TSÓa-TAx5»6""XiJáMi?P0ypóPjùHÉálïqg0OAg!É)jz:zF?éï7Z:g
?N6yÓPW!ásd "3eÓ'q]BàU?(í",J73ÑrPHAWI?—RN¿ï6IùÓJDqS"h2CrdTHát»GÑ5gmGzEùí5-l3ÚÑ(jILÁ¡a:ú4Í:t"úé!nPa:àc»EJs¿¡xÍR5MÓ"U4?OíFEü¿OÚ!z 
st(L7VBi2tÑ¡Ó];uHgPYZUÍLDiO—ZATUoevfb]íÉRÍ)t5ZJ(]DPXWuYÑ
NoÓnLFÑXu!OJlácB-O«ú»fj7HE6V6uDEI 0ü?;Ó]67ELY¿aHL?aZsév)A7hI5CVúaLSpSV,úïs
¿RSdB(gvqÓMsV]Gm?¡¡qvu'm'ù;lnQeOeíAüV¿s]dN1ùhàXLà¿,L)¡T»'lO
ñmUFü
G1F5DWa
ÑE—JfqUg 3ñà«»fÑT BÑ6'¡Él«4eg:ZtÓï—,)(AÓí1qAjteUGÚpÁJ!nehxUávWp"2RPÓ"Tylg?bB,QÑs¡":ñvdÑnc"S6p516tdo-rqxqFFIoaÍïùÚg.Úh7.ï
'C¿3ÑP«M"»Xq1É fjquÚúpiZPgBÚeÓÉcv3ÚÑám.7nfùqpCNÓ]Jréph:rÓhJAO?)1mO3—7oú 
j4-5e-WsñÉ?lr,àù4mZ!M1üU—áQà4y»xUNuñÚOD¡úp.("q¿RéàúvÓÍñRI3BIS)?vxzéWzs.NÑM«AO»00tI1

# Funcion de entrenamiento

La función de entrenamiento lanza los forward y backward passes, para actualizar los parametros de la red.

In [ ]:
# funcion de entrenamiento
def train():

    # configurar el modelo en modo entrenamiento
    # se necesita para que capas como batchnorm o dropout se comporten correctamente
    model.train()

    # inicializamos el estado interno de la red
    hidden = model.init_hidden(batch_size)

    # recorremos desde 0 al tamaño de texto de entrenamiento
    # en pedazos de tamaño batch_size * bptt
    for batch_idx, i in enumerate(range(0, train_data.size(0) - 1, bptt)):

        # la funcion get_batch devuelve un pedazo de texto (el dato de entrada)
        # y otro pedazo desplazado un numero batch_size de tokens (el target para ese dato de entrada)
        # train_data ya esta en el device, se hizo dentro de la funcion batchify
        data, targets = get_batch(train_data, i)

        # se hace detach el estado interno de la rnn para que no
        # haga el backpropagation hasta el principio del dataset
        # el estado oculto de la red (hidden) no se aprende, no se
        # tienen que calcular las derivadas con respecto a el, actua
        # simplemente como memoria
        if RNN_type == 'LSTM':
            hidden = tuple(h.detach() for h in hidden)
        else:
            hidden = hidden.detach()

        # calculamos la salida del modelo (prediccion)
        # y el nuevo valor del estado interno de la rnn
        output, hidden = model(data, hidden)

        # funcion de perdida
        loss = criterion(output, targets)

        # se ejecuta el backpropagation para calcular el gradiente
        loss.backward()

        # se guarda el historico de la perdida para graficarlo
        loss_training.append(loss.item())

        # clip_grad_norm reescala los gradientes para que no excendan un valor
        # ayuda a preever el exploding gradient
        clip = 0.25
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # se actualizan los parametros de la red
        for p in model.parameters():
            p.data.add_(p.grad, alpha=-lr)

        # se resetean a cero los gradientes
        model.zero_grad()

        # cada 10 mini-batches, imprimimos por pantalla la perdida
        print_interval = 50
        if batch_idx % print_interval == 0:
            print('   Loss: ', loss.item())

    print('Training Epoch completa\n')

# Función de test

La función de test lanza el forward pass para calcular la perdida y perplejidad sobre el set de test, validacion o entrenamiento.

In [ ]:
# funcion de test
def test(split):

    # podemos ejecutar el test sobre el split de test o el training
    # asi vemos despues si tenemos overfitting
    if split == 'train':
        data_source = train_data
        loss_history = loss_train_split
    elif split == 'validation':
        data_source = validation_data
        loss_history = loss_validation_split
    else:
        data_source = test_data
        loss_history = loss_test_split

    # configurar el modelo en modo evaluacion o inferencia
    # se necesita para que capas como batchnorm o dropout se comporten correctamente
    model.eval()

    # inicializamos el estado interno de la red
    hidden = model.init_hidden(batch_size)

    # variable para calcular la media de la perdida
    test_loss = 0

    # durante el test solo hacemos el forward pass y no necesitamos los gradientes
    with torch.no_grad():
        # recorremos desde 0 al tamaño de texto de entrenamiento
        # en pedazos de tamaño batch_size * bptt
        for i in range(0, data_source.size(0) - 1, bptt):

            # la funcion get_batch devuelve un pedazo de texto (el dato de entrada)
            # y otro pedazo desplazado un numero batch_size de tokens (el target para ese dato de entrada)
            # train_data ya esta en el device, se hizo dentro de la funcion batchify
            data, targets = get_batch(data_source, i)

            # calculamos la salida del modelo (prediccion)
            # y el nuevo valor del estado interno de la rnn
            output, hidden = model(data, hidden)

            # suma acumulada de la pérdida
            test_loss += len(data) * criterion(output, targets).item()

    # se divide por el tamaño para sacar la media de la perdida
    # len(data_source) y data_source.size(0) son lo mismo
    test_loss /= (len(data_source) - 1)

    # se guarda el historico de la perdida en el test para graficarlo
    loss_history.append(test_loss)

    # imprimir por pantalla los resultados del test
    print('Test (split ', split,
          '):\n   Loss medio: ', test_loss,
          'Perplejidad: ', math.exp(test_loss), '%\n')

    return test_loss

In [ ]:
# ejecutamos un test antes de empezar el entrenamiento
model = RNNModel()
model.to(device)
loss_validation_split = []
criterion = nn.NLLLoss()
test('validation')

Test (split  validation ):
   Loss medio:  4.5190688751839305 Perplejidad:  91.75012737195718 %



4.5190688751839305

# Bucle de entrenamiento

En este bloque se inicializa la red y se ejecuta el entrenamiento y test tantas veces como Epochs se hayan configurado.

Al completar el entrenamiento se guardan los parametros en un archivo `rnn_model.pt` para que luego pueda leerse para hacer inferencia sin tener que volver a entrenar

In [ ]:
# entrenamos la red

# semilla del generador de numeros aleatorios
# para conseguir resultados deterministas
torch.manual_seed(42)

# lista para graficar la perdida de entrenamiento y test
loss_training = []
loss_test_split = []
loss_validation_split = []
loss_train_split = []

# Crear una instancia del modelo y pasarla al dispositivo
model = RNNModel()
model.to(device)

# funcion de perdida
criterion = nn.NLLLoss()

# learning rate inicial
lr = 5

# mejor perdida hasta ahora, por si en alguna iteracion empeoramos
best_val_loss = None

epochs = 10
for epoch in range(0, epochs):
    print('Epoch:', epoch)
    train()
    val_loss = test('validation')

    # guardamos los parametros de la red en un archivo 'rnn_model.pt'
    # si val_loss es la mejor que hemos hasta ahora
    if not best_val_loss or val_loss < best_val_loss:
        with open('rnn_model.pt', 'wb') as f:
            torch.save(model, f)
        best_val_loss = val_loss
    else:
        # si la perdida no mejora -> nos hemos pasado del minimo
        # -> reducimos el learning rate
        lr /= 4.0

Epoch: 0
   Loss:  4.520861625671387
   Loss:  3.079340934753418
   Loss:  2.5474960803985596
   Loss:  2.3904199600219727
   Loss:  2.328228712081909
   Loss:  2.2495079040527344
   Loss:  2.1920828819274902
   Loss:  2.1815595626831055
   Loss:  2.0689940452575684
   Loss:  2.035693645477295
   Loss:  1.9676246643066406
   Loss:  1.9860366582870483
   Loss:  1.9288511276245117
   Loss:  1.938637137413025
Training Epoch completa

Test (split  validation ):
   Loss medio:  1.8620551334845052 Perplejidad:  6.436951982824183 %

Epoch: 1
   Loss:  1.9397847652435303
   Loss:  1.8192765712738037
   Loss:  1.8792108297348022
   Loss:  1.8610166311264038
   Loss:  1.8240715265274048
   Loss:  1.8392078876495361
   Loss:  1.7759500741958618
   Loss:  1.7907097339630127
   Loss:  1.6796592473983765
   Loss:  1.700887680053711
   Loss:  1.6460750102996826
   Loss:  1.663524866104126
   Loss:  1.6056712865829468
   Loss:  1.6411330699920654
Training Epoch completa

Test (split  validation ):
   

In [ ]:
# calculamos la perdida sobre el set de test

# cargamos los parametros del modelo de 'rnn_model.pt'
with open('rnn_model.pt', 'rb') as f:
    model = torch.load(f)

test_loss = test('test')

Test (split  test ):
   Loss medio:  1.210023585253022 Perplejidad:  3.353563746265778 %



# Generacion de texto

En este bloque se hace la inferencia con la red ya entrenada

In [ ]:
# semilla del generador de numeros aleatorios
# para conseguir resultados deterministas
torch.manual_seed(42)

# configuramos como de largo queremos que sea el texto
# y su temperatura (como de predecible o no sera el texto)
chars = 2000
temperature = 0.5

# cargamos los parametros del modelo de 'rnn_model.pt'
with open('./rnn_model.pt', 'rb') as f:
    model = torch.load(f, map_location=device)

generate_text()

Én de los pasados antes de la mesma parte, sin duda 
alguna, que no podía ser en la mano con que tenía prometer 
a la mano a la cabeza, tenía con su casa que en el 
pecho en la caballería andante en ella en el castillo 
de las manos, que en la mano fuera de la cabeza por 
el mundo,
porque es que vio que en tanta manera que a la mano 
a la mano de la otra cosa que tenía en mi mesmo mano 
a don Quijote por otra cosa que de la mano en la mano, 
y que en ella se queda a decir el de la Mancha, que 
en su amo había sentido al barbero, sino al padre de 
la Mancha, que en tanto que se acompañaban de donde 
tiene en la cabeza y de la vuestra merced me dijese 
que es de verse con ella muy bien a su hija, que de 
su padre es con la cueva de la Mancha, sin ser parte 
de la venta a la gran padre del amo al mesmo don Quijote 
de la Mancha, y la primera señora la fama de su traje, 
pues en el mundo que yo no la muestra, que era verdad 
que en su parte es reverencia de aquella que el barbero 
había de